In [1]:
import numpy as np
import matplotlib.pyplot as plt
import random
!pip install numba 
from numba import njit
# defining simple lattice functions to place n particles in a box of length L 

def simple_lattice_2D(L,n):
     positions = []
     N = int(np.sqrt(n))
     separation = L / N
     for i in range(0,N):
            for j in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                positions.append(list([x,y]))
     return positions


def simple_lattice_3D(L,n):
    positions = []
    N = int(np.cbrt(n))
    separation = L / N
    for i in range(0,N):
        for j in range(0,N):
            for k in range(0,N):
                x = (i + 0.5) * separation
                y = (j + 0.5) * separation
                z = (k + 0.5) * separation
                positions.append((x,y,z))
    return positions

def particle_diameter_3D(L, n, vol_fraction):
    volume_per_particle = (L**3 * vol_fraction) / n
    diameter = 2*(3 * volume_per_particle / (4 * np.pi))**(1/3)
    return diameter
def particle_diameter_2D(L,n, area_fraction):
    area_per_particle = (L**2 * area_fraction) / n
    diameter = 2*(area_per_particle / np.pi)**(1/2)
    return diameter

x_coords, y_coords = zip(*simple_lattice_2D(1,100))
plt.figure( facecolor=(1, 1, 1, 0.3))
plt.plot(x_coords, y_coords, 'o', markersize = 23.5)
plt.xlim(0,1)
plt.ylim(0,1)
plt.xticks([])
plt.yticks([])
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Simple Lattice in 2D', fontsize = 16)
plt.savefig('simple_lattice_2D')

def monte_carlo_2D_np(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # Random displacement along x or y
    axis = random.randint(0, 1)       # 0=x, 1=y
    direction = 1 if random.randint(0, 1) == 0 else -1
    new_positions[particle, axis] += direction * displacement
    
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move
    
L = 4
n = 4 
vol_fraction = 0.1
no_density = n / (L**2)
diameter = particle_diameter_2D(L, n, vol_fraction)



In [2]:
# create stepwise function to move random particle in random direction by displacement amount specified
#also account for the periodic boundary conditions inside the movement 
import random
def monte_carlo_2D(positions, displacement, L, diameter):
    positions = np.array(positions, dtype=float)  # ensure numeric array
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    #random angle for movement and then calculate x and y changes
    angle = random.uniform(0, 2 * np.pi)
    x_change = displacement * np.cos(angle)
    y_change = displacement * np.sin(angle)
    new_positions[particle, 0] += x_change
    new_positions[particle, 1] += y_change
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 2)
    
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    
    distances = np.linalg.norm(diffs, axis=1)
    distances[particle] = np.inf  # ignore self-distance
    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move



In [3]:
#list of provided parameters - good starting point


L = 1
n = 100
vol_fraction = 0.72
no_density = n / (L**2)
diameter = particle_diameter_2D(L, n, vol_fraction)
pos_0 = simple_lattice_2D(L,n)

pos_new = monte_carlo_2D(pos_0, 0.0001, L, diameter)

if np.array_equal(pos_new, pos_0):
    print("Move rejected")
else:
    print("Move accepted")
  
#particles free to move with monte carlo function for 2D
#code to find the optimum displacement value to achieve 25% - 50% acceptance rate
acceptance_count = 0
pos_ref = pos_0
for i in range(0,1000):
    pos_new = monte_carlo_2D(pos_ref, 0.0043,L, diameter)
    if np.array_equal(pos_new, pos_ref):
        pass #move rejected
    else:
        acceptance_count += 1 #move accepted
        pos_ref = pos_new
print(acceptance_count)

#gives rougly 39% acceptance rate for the suggested parameters

Move accepted
426


In [4]:
#define total energy calculation function

def total_energy(positions, L, diameter):
    positions = np.array(positions, dtype=float)
    N = len(positions)
    total_energy = 0.0
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = np.linalg.norm(diff)
            
            # Simple hard-sphere potential
            if distance < diameter:
                total_energy += np.inf  # Infinite energy for overlap
            else:
                total_energy += 0.0  # No interaction otherwise
                
    return np.array(total_energy)

# define function to calculate the change in energy after a single move
def energy_change(positions, initial_energy, new_positions):
    new_coords = np.any(new_positions != positions, axis=1)
    moved_particle = np.where(new_coords)[0][0]
    new_energy = initial_energy
    for i in range(0,len(positions)):
        if i == moved_particle:
            continue  # Skip self-interaction
        # Compute distance with minimum image convention
        diff_old = positions[moved_particle] - positions[i]
        diff_old = diff_old - L * np.round(diff_old / L)
        distance_old = np.linalg.norm(diff_old)
        
        diff_new = new_positions[moved_particle] - new_positions[i]
        diff_new = diff_new - L * np.round(diff_new / L)
        distance_new = np.linalg.norm(diff_new)
        
        # Simple hard-sphere potential
            
        if distance_new < diameter and distance_old >= diameter:
            new_energy = np.inf + initial_energy # add infinite energy if moving into overlap
        if distance_new >= diameter and distance_old < diameter:
            new_energy = -np.inf + initial_energy # Remove infinite energy if moving out of overlap
        else:
            new_energy = 0.0 + initial_energy # no overlap either before or after
    return new_energy

In [5]:
#test initial energy and energy change functions
initial_energy = total_energy(pos_0, L, diameter)
pos_new = monte_carlo_2D(pos_0, 0.0043,L,diameter)
delta_E = energy_change(np.array(pos_0), initial_energy, pos_new)
print(initial_energy)
print(delta_E)
#i think this works, as E should always be 

0.0
0.0


In [6]:
#defining the metropolis algorithm - will have to change the monte carlo to accept even if center of spheres are within a radius
def metropolis(positions,initial_energy, delta_E_func, stepwise_function, displacement, L, diameter, k_B, T):
    positions = np.array(positions)
    new_positions = stepwise_function(positions, displacement, L , diameter)
    delta_E = delta_E_func(positions, initial_energy, new_positions)
    if delta_E > 0:
        n == random.rand(0,1)
        if n < np.exp(-delta_E / (k_B * T)):#how do we find T
            return  new_positions
        else:
            return positions
    else:
        return positions
# this should allow you to input the initial energy, positions, displacement, L Boltzman constant and temperature 
#then will perform a montecarlo step to the system to move particle
#calculate the energy after the move and accept if energy is lowered
#if energy is the same or more then it is accepted if the metropolis function is less than the random number generated


In [7]:
#define the pair correlation function
#try delta_r as approx 0.1 times the diameter as initial trial
@njit
def pair_correlation_2D(positions, diameter, L, bins=100):
    N = len(positions) # no.of particles
    hist = np.zeros(bins, dtype=float) # create histogram array for each bin
    delta_r = (L / 2) / bins  # bin width
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = (diff[0]**2 + diff[1]**2)**0.5
            
            # Determine which bin this distance falls into
            bin_index = int(distance / delta_r)
            if bin_index < bins:
                hist[bin_index] += 1  # count each pair once
            
    r = ((np.arange(bins) + 0.5) * delta_r) / diameter # midpoint of each bin for average value of r 
    area_density = N / (L**2)
    n_ideal = 2 * np.pi * r * delta_r * area_density * N # ideal gas distribution

    g_r = hist / n_ideal
    return r, g_r




In [8]:
#trial of pair correlation function
N = 100
L = 1
eta_68 = 0.68
#equilibration run for area fration = 0.68
diameter_68 = particle_diameter_2D(L, N, eta_68)
pos_0_68 = np.array(simple_lattice_2D(L,N))
pos_68 = pos_0_68.copy()
r_68, g_r_68 = pair_correlation_2D(pos_68,diameter_68, L)
r_initial_68, g_r_initial = pair_correlation_2D(pos_0_68, diameter_68, L) # initial position before equilibrating
#run monte carlo 10**4 times
g_r_total_eq = []

for i in range(0, 4*10 **4 + 1):
    pos_68 = monte_carlo_2D(pos_68, 0.0043, L, diameter_68)
    if i % 10000 == 0:
        r, g_r = pair_correlation_2D(pos_68,diameter_68, L, bins = 100)
        g_r_total_eq.append(g_r)

x2_coords = r
y1_coords = g_r_total_eq[0]
y2_coords = g_r_total_eq[1]
y3_coords = g_r_total_eq[2]
y4_coords = g_r_total_eq[3]
y5_coords = g_r_total_eq[4]
plt.figure()
plt.plot(x2_coords, y1_coords, label='After 0 moves')
plt.plot(x2_coords, y2_coords, label='After 10000 moves')
plt.plot(x2_coords, y3_coords, label='After 20000 moves')
plt.plot(x2_coords, y4_coords, label='After 30000 moves')
plt.plot(x2_coords, y5_coords, label='After 40000 moves')
plt.savefig('equillibration_2D.png')


x_coords, y_coords = zip(*pos_68)
x_coords = list(x_coords)
y_coords = list(y_coords)
x_original = x_coords.copy()
y_original = y_coords.copy()
#adding a shiftedcopy to left hand side of box
x_coords += [x - 1 for x in x_original]
y_coords += y_original
#adding shifted to right side of box
x_coords += [x + 1 for x in x_original]
y_coords += y_original
#adding shifte to top of box
x_coords += x_original
y_coords += [y + 1 for y in y_original]
#adding shifted to bottom of box
x_coords += x_original
y_coords += [y - 1 for y in y_original]

plt.figure(facecolor = (1,1,1,0.3))
plt.plot(x_coords, y_coords, 'o', markersize = 23.5)
plt.xlim(0,1)
plt.ylim(0,1)
plt.xticks([])
plt.yticks([])
plt.gca().set_aspect('equal', adjustable='box')
plt.title('Equillibrated Hard Disk System', fontsize = 16)
plt.savefig('simple_lattice_2D_equillibrated.png')

In [9]:
#averagine the g_r values after equillibration
#use position from previous code block
g_r_total_68 = []
#run through the monte carlo steps again    
for i in range(0, 5*10 **5 + 1):
    pos_68 = monte_carlo_2D(pos_68, 0.0043, L, diameter_68)
    if i % 1000 == 0:
        r_68, g_r_68 = pair_correlation_2D(pos_68,diameter_68, L, bins = 100)
        g_r_total_68.append(g_r_68)

x_68 = r_68
y_68 = np.mean(g_r_total_68, axis=0)    

#don't run any more code in this box as takes approx 20 seconds to run
#runs way faster with numba acceleration so is fine now
#now calculate for different values of area fraction (0.69, 0.70, 0.71, 0.72)
eta_69 = 0.69
diameter_69 = particle_diameter_2D(L, N, eta_69)
pos_0_69 = np.array(simple_lattice_2D(L,N))
pos_69 = pos_0_69.copy()
r_69, g_r_69 = pair_correlation_2D(pos_69,diameter_69, L)
g_r_total_69 = []
#run through the monte carlo steps again
for i in range(0, 4*10 **4 + 1):
    pos_69 = monte_carlo_2D(pos_69, 0.0043, L, diameter_68)
for i in range(0, 5*10 **5 + 1):
    pos_69 = monte_carlo_2D(pos_69, 0.0043, L, diameter_69)
    if i % 1000 == 0:
        r_69, g_r_69 = pair_correlation_2D(pos_69,diameter_69, L, bins = 100)
        g_r_total_69.append(g_r_69)
x_69 = r_69
y_69 = np.mean(g_r_total_69, axis=0)    

eta_70 = 0.70
diameter_70 = particle_diameter_2D(L, N, eta_70)
pos_0_70 = np.array(simple_lattice_2D(L,N))
pos_70 = pos_0_70.copy()
r_70, g_r_70 = pair_correlation_2D(pos_70,diameter_70, L)
g_r_total_70 = []
#run through the monte carlo steps again
for i in range(0, 1*10 **5 + 1):
    pos_70 = monte_carlo_2D(pos_70, 0.0043, L, diameter_70)
for i in range(0, 5*10 **5 + 1):
    pos_70 = monte_carlo_2D(pos_70, 0.0043, L, diameter_70)
    if i % 1000 == 0:
        r_70, g_r_70 = pair_correlation_2D(pos_70,diameter_70, L, bins = 100)
        g_r_total_70.append(g_r_70)
x_70 = r_70
y_70 = np.mean(g_r_total_70, axis=0)            

eta_71 = 0.71
diameter_71 = particle_diameter_2D(L, N, eta_71)
pos_0_71 = np.array(simple_lattice_2D(L,N))
pos_71 = pos_0_71.copy()
r_71, g_r_71 = pair_correlation_2D(pos_71,diameter_71, L)
g_r_total_71 = []
#run through the monte carlo steps again
for i in range(0, 1*10 **5 + 1):
    pos_71 = monte_carlo_2D(pos_71, 0.0043, L, diameter_71)
for i in range(0, 5*10 **5 + 1):                    
    pos_71 = monte_carlo_2D(pos_71, 0.0043, L, diameter_71)
    if i % 1000 == 0:
        r_71, g_r_71 = pair_correlation_2D(pos_71,diameter_71, L, bins = 100)
        g_r_total_71.append(g_r_71)
x_71 = r_71
y_71 = np.mean(g_r_total_71, axis=0)    

eta = 0.72
diameter_72 = particle_diameter_2D(L, N, eta)
pos_0_72 = np.array(simple_lattice_2D(L,N))
pos_72 = pos_0_72.copy()
r_72, g_r_72 = pair_correlation_2D(pos_72,diameter_72, L)
g_r_total_72 = []
#run through the monte carlo steps again
for i in range(0, 1*10 **5 + 1):
    pos_72 = monte_carlo_2D(pos_72, 0.0043, L, diameter_72)
for i in range(0, 5*10 **5 + 1):                    
    pos_72 = monte_carlo_2D(pos_72, 0.0043, L, diameter_72)
    if i % 1000 == 0:
        r_72, g_r_72 = pair_correlation_2D(pos_72,diameter_72, L, bins = 100)
        g_r_total_72.append(g_r_72)
x_72 = r_72
y_72 = np.mean(g_r_total_72, axis=0)    


In [10]:
#make a pretty graph for 2D average g(r)
#calculate some errors too

g_err_68 = np.std(g_r_total_68, axis = 0) / np.sqrt(len(g_r_total_68))
x_err  = (L/2) / 200 # bin width / 2
g_err_69 = np.std(g_r_total_69, axis = 0) / np.sqrt(len(g_r_total_69))
g_err_70 = np.std(g_r_total_70, axis = 0) / np.sqrt(len(g_r_total_70))
g_err_71 = np.std(g_r_total_71, axis = 0) / np.sqrt(len(g_r_total_71))
g_err_72 = np.std(g_r_total_72, axis = 0) / np.sqrt(len(g_r_total_72))
#could try to find a model to fit - might be a decaying exponential? / don't know how to do this though

plt.figure()
plt.plot(x_68, y_68, color = 'black', linewidth = 0.5, label = 'Area Fraction = 0.68')
#plt.errorbar(x_68, y_68, yerr= g_err_68, xerr = x_err, fmt = 'o',color = 'blue', ecolor = 'navy', capsize = 2, markersize = 1, label = 'area fraction = 0.68')
#plt.errorbar(x_69, y_69, yerr= g_err_69, xerr = x_err, fmt = 'o',color = 'orange', ecolor = 'red', capsize = 2, markersize = 1, label = 'area fraction = 0.69')
#plt.errorbar(x_70, y_70, xerr = x_err, yerr = g_err_70, fmt = 'o',color = 'green', ecolor = 'darkgreen', capsize = 2, markersize = 1, label = 'area fraction = 0.70')
#plt.errorbar(x_71, y_71, xerr = x_err, yerr = g_err_71, fmt = 'o',color = 'red', ecolor = 'darkred', capsize = 2, markersize = 1, label = 'area fraction = 0.71')
#plt.errorbar(x_72, y_72, xerr = x_err, yerr = g_err_72, fmt = 'o',color = 'purple', ecolor = 'indigo', capsize = 2, markersize = 1, label = 'area fraction = 0.72')
#plt.plot(x_69, y_69, color = '', linewidth = 0.5, label = 'area fraction = 0.69')
plt.plot(x_70, y_70, color = 'green', linewidth = 0.5, label = 'Area Fraction = 0.70')
#plt.plot(x_71, y_71, color = 'red', linewidth = 0.5, label = 'area fraction = 0.71')
plt.plot(x_72, y_72, color = 'purple', linewidth = 0.5, label = 'Area Fraction = 0.72')
plt.xlabel('r / $\sigma$')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('average_g(r).png')


<>:23: SyntaxWarning: invalid escape sequence '\s'
<>:23: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_28206/2989302950.py:23: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('r / $\sigma$')


In [11]:
#extend the functions to 3D
#define the monte carlo function for stepwise in 3D - still using the hard sphere model so no overlaps
def monte_carlo_3D(positions, displacement, L, diameter):
    N = len(positions)
    
    # Pick a random particle
    particle = random.randint(0, N - 1)
    
    # Copy positions for trial move
    new_positions = positions.copy()
    
    # find 2 random angles for movement in 3D
    cos_theta = np.random.uniform(-1, 1)
    theta = np.arccos(cos_theta)
    phi = random.uniform(0, 2 * np.pi)
    x_change = displacement * np.sin(theta) * np.cos(phi)
    y_change = displacement * np.sin(theta) * np.sin(phi)
    z_change = displacement * cos_theta
    new_positions[particle, 0] += x_change
    new_positions[particle, 1] += y_change
    new_positions[particle, 2] += z_change
    # Apply periodic boundary conditions
    new_positions[particle] %= L
    # Compute distances from the moved particle to all others
    diffs = new_positions - new_positions[particle]         # shape (N, 3)
    # Minimum image convention for periodic boundaries
    diffs = diffs - L * np.round(diffs / L)
    distances = (diffs**2).sum(axis=1)**0.5
    distances[particle] = np.inf  # ignore self-distance    
    # Check overlap
    if np.any(distances < diameter):
        return positions  # reject move
    else:
        return new_positions  # accept move
    
#define the pair correlation function for 3D
@njit
def pair_correlation_3D(positions, diameter, L, bins=100):
    N = len(positions) # no.of particles
    hist = np.zeros(bins, dtype=float) # create histogram array for each bin
    delta_r = (L / 2) / bins  # bin width
    
    for i in range(N):
        for j in range(i + 1, N):
            # Compute distance with minimum image convention
            diff = positions[i] - positions[j]
            diff = diff - L * np.round(diff / L)
            distance = (diff[0]**2 + diff[1]**2 + diff[2]**2)**0.5
            
            # Determine which bin this distance falls into
            bin_index = int(distance / delta_r)
            if bin_index < bins:
                hist[bin_index] += 1  # count each pair once
            
    r = ((np.arange(bins) + 0.5) * delta_r) / diameter # midpoint of each bin for average value of r 
    volume_density = N / (L**3)
    n_ideal = 4 * np.pi * r**2 * delta_r * volume_density * N # ideal gas distribution

    g_r = 10 * hist / n_ideal
    return r, g_r

def generate_fcc(L, nx, ny=None, nz=None):
    """
    Generate an FCC lattice inside a cubic box L.
    nx,ny,nz = number of conventional FCC cells per axis (integers).
    Returns positions array with shape (4*nx*ny*nz, 3).
    """
    if ny is None: ny = nx
    if nz is None: nz = nx
    a = L / nx  # lattice constant such that nx cells fill the box
    # conventional FCC basis (in units of a)
    basis = np.array([
        [0.0, 0.0, 0.0],
        [0.5, 0.5, 0.0],
        [0.5, 0.0, 0.5],
        [0.0, 0.5, 0.5]
    ])
    positions = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                origin = np.array([i, j, k], dtype=float) * a
                for b in basis:
                    positions.append(origin + b * a)
    return np.array(positions, dtype=float)

# Example: make 10x10x10 conventional FCC cells in box L
L = 1.0
nx = 10
pos = generate_fcc(L, nx)  # gives 4 * 10^3 = 4000 particles


In [12]:
#need to test to find when approx 25%-50% acceptance rate for 3D
L = 1
vol_fraction_3D = 0.58
d = particle_diameter_3D(L, 864, vol_fraction_3D)
pos_3D = np.array(generate_fcc(L, 6), dtype=float)
acceptance_count_3D  = 0 
for i in range(0,1000):
    pos_new_3D = monte_carlo_3D(pos_3D, 0.01,L, d)
    if pos_new_3D is pos_3D:
        pass #move rejected
    else:
        acceptance_count_3D += 1 #move accepted
        pos_3D = pos_new_3D
print(acceptance_count_3D)



392


In [13]:
#testing out the 3D pair correlation function
#lols chatgpt thought this would take 30-60 minutes with just numpy
#tr with fcc cubic as initial structure sine couldnt go above 0.5 vol fraction with simple cubic

L = 1
N = 864
positions = np.array(generate_fcc(L, 6), dtype=float)

vol_fraction = 0.5
d_50 = particle_diameter_3D(L, N, vol_fraction)
pos_3D_50 = positions.copy()
r_3D, g_r_3D_50 = pair_correlation_3D(positions, d_50, L, bins=100)
g_r_3D_50 = []
#equillibration run
for i in range(0, 7*10 **5 + 1):
    pos_3D_50 = monte_carlo_3D(pos_3D_50, 0.01 , L, d_50)
    if i % 10000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_50,d_50, L, bins = 100)
        g_r_3D_50.append(g_r_new_3D)
x_coords_3D = r_3D
y_coords_3D_50 = np.mean(g_r_3D_50, axis=0)
plt.figure()
plt.plot(x_coords_3D, y_coords_3D_50)
plt.savefig('equillibration_3D.png')

#continue from equillibration to get average g(r)
g_r_3D_50 = []
#run through the monte carlo steps again
for i in range(0, 5*10 **5 + 1):
    pos_3D_50 = monte_carlo_3D(pos_3D_50, 0.01, L, d_50)
    if i % 1000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_50, d_50, L, bins = 100)
        g_r_3D_50.append(g_r_new_3D)
y_coords_3D_50 = np.mean(g_r_3D_50, axis=0)

In [14]:
#complete g(r) function for different volume fractions in 3D
vol_frac_55 = 0.55
d_55 = particle_diameter_3D(L, N, vol_frac_55)
pos_3D_55 = positions.copy()
r_3D, g_r_3D_55 = pair_correlation_3D(positions, d_55, L, bins=100)
g_r_3D_55 = []
#equillibration run
for i in range(0, 7*10 **5 + 1):
    pos_3D_55 = monte_carlo_3D(pos_3D_55, 0.01 , L, d_55)
    if i % 10000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_55,d_55, L, bins = 100)
        g_r_3D_55.append(g_r_new_3D)

#continue from equillibration to get average g(r)
g_r_3D_55 = []
#run through the monte carlo steps again
for i in range(0, 5*10 **5 + 1):
    pos_3D_55 = monte_carlo_3D(pos_3D_55, 0.01, L, d_55)
    if i % 1000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_55, d_55, L, bins = 100)
        g_r_3D_55.append(g_r_new_3D)
y_coords_3D_55 = np.mean(g_r_3D_55, axis=0)


vol_fraction = 0.58
d_58 = particle_diameter_3D(L, N, vol_fraction)
pos_3D_58 = positions.copy()
r_3D, g_r_3D_58 = pair_correlation_3D(positions, d_58, L, bins=100)
g_r_3D_58 = []
#equillibration run
for i in range(0, 7*10 **5 + 1):
    pos_3D_58 = monte_carlo_3D(pos_3D_58, 0.01, L, d_58)
    if i % 10000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_58,d_58, L, bins = 100)
        g_r_3D_58.append(g_r_new_3D)

#continue from equillibration to get average g(r)
g_r_3D_58 = []
#run through the monte carlo steps again
for i in range(0, 5*10 **5 + 1):
    pos_3D_58 = monte_carlo_3D(pos_3D_58, 0.01, L, d_58)
    if i % 1000 == 0:
        r_3D_new, g_r_new_3D = pair_correlation_3D(pos_3D_58, d_58, L, bins = 100)
        g_r_3D_58.append(g_r_new_3D)
y_coords_3D_58 = np.mean(g_r_3D_58, axis=0)

In [15]:
plt.figure()
plt.plot(x_coords_3D, y_coords_3D_50, label = 'vol frac = 0.50')
plt.plot(x_coords_3D, y_coords_3D_55, label = 'vol frac = 0.55')
plt.plot(x_coords_3D, y_coords_3D_58, label = 'vol frac = 0.58')
plt.xlabel('r / $\sigma$')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('average_g(r)_3D.png')

<>:5: SyntaxWarning: invalid escape sequence '\s'
<>:5: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_28206/4173109073.py:5: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('r / $\sigma$')


In [16]:
#plot one figure with two axis to have 2D and 3D g(r) on same graph
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec



fig = plt.figure(figsize=(8, 6))

# Create two stacked axes with no vertical spacing
gs = gridspec.GridSpec(2, 1, height_ratios=[1, 1], hspace=0)

# -------------------------
# TOP MAIN AXIS
# -------------------------
ax_top = fig.add_subplot(gs[0])
ax_top.plot(x_68, y_68, color = 'red', linewidth = 0.6, label = 'Area Fraction = 0.68')
ax_top.plot(x_72, y_72, color = 'blueviolet', linewidth = 0.6, label = 'Area Fraction = 0.72')
ax_top.legend(loc='upper right')
ax_top.set_xticks([0,1,2,3,4,5])
ax_top.set_yticks([0,0.1,0.2])
ax_top.set_xlim(0,5)
ax_top.set_ylim(0,0.28)
ax_top.set_ylabel("g(r) - 2D ", fontsize=14)

# -------------------------
# INSET AXIS (ZOOM VIEW)
# -------------------------
ax_inset = inset_axes(ax_top, width="30%", height="40%", loc="upper center")

# zoom region
xmin, xmax = 1.5, 2.1
ymin, ymax = 0, 0.1
ax_inset.set_xlim(xmin, xmax)
ax_inset.set_ylim(ymin, ymax)

ax_inset.plot(x_68,y_68, color = 'red', linewidth = 0.6)
ax_inset.plot(x_72, y_72, color = 'blueviolet', linewidth = 0.6)
ax_inset.set_xticks([])
ax_inset.set_yticks([])
# Draw bounding box and dashed connector lines
mark_inset(ax_top, ax_inset, loc1=2, loc2=4, fc="none", ec="black", linestyle="--", alpha = 0.5)

# -------------------------
# BOTTOM AXIS - 3D
# -------------------------
ax_bottom = fig.add_subplot(gs[1], sharex=ax_top)

ax_bottom.plot(x_coords_3D, y_coords_3D_50, color = 'navy', linewidth = 0.6, label = 'Volume Fraction = 0.50')
ax_bottom.plot(x_coords_3D, y_coords_3D_58, color = 'darkgreen', linewidth = 0.6, label = 'Volume Fraction = 0.58')

ax_bottom.set_xlabel("r / $\sigma$")
ax_bottom.set_ylabel("g(r) - 3D ")
ax_bottom.legend(loc='upper right')
ax_bottom.set_xticks([0,1,2,3,4,5])
ax_bottom.set_yticks([0,0.1,0.2,0.3])
ax_bottom.set_xlim(0,5)
ax_bottom.set_ylim(0,0.32)
#inset for bottom axis

ax_inset = inset_axes(ax_bottom, width="30%", height="40%", loc= 'upper center' )
xmin, xmax = 1.3, 2.0
ymin, ymax = 0, 0.1
ax_inset.set_xlim(xmin, xmax)
ax_inset.set_ylim(ymin, ymax)
ax_inset.set_xticks([])
ax_inset.set_yticks([])


ax_inset.plot(x_coords_3D, y_coords_3D_50, color = 'navy', linewidth = 0.6)
ax_inset.plot(x_coords_3D, y_coords_3D_58, color = 'darkgreen', linewidth = 0.6)

mark_inset(ax_bottom, ax_inset, loc1=2,loc2 = 4, fc="none", ec="black", linestyle="--", alpha = 0.5)
# Remove top axis x tick labels so they don't overlap
plt.setp(ax_top.get_xticklabels(), visible=False)
plt.show()

plt.savefig('2D_and_3D_g(r).png') 

<>:53: SyntaxWarning: invalid escape sequence '\s'
<>:53: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_28206/4263737930.py:53: SyntaxWarning: invalid escape sequence '\s'
  ax_bottom.set_xlabel("r / $\sigma$")
/tmp/ipykernel_28206/4263737930.py:77: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [17]:
from mpl_toolkits.mplot3d import Axes3D 
x,y,z = zip(*pos_3D_58)
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

ax.scatter(x, y, z)

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')
ax.set_xlim(0,1)
ax.set_ylim(0,1)
ax.set_zlim(0,1)
plt.title('Equillibrated Hard Sphere System (Vol Fraction = 0.58)')
plt.savefig('simple_lattice_3D_equillibrated.png')

x1,y1,z1 = zip(*positions)
fig = plt.figure()
ax = fig.add_subplot(111, projection = '3d')
ax.scatter(x1,y1,z1)
plt.savefig('FCC Lattice 3D.png')

In [18]:
#create function that works for 2 different sized particles in lattice 
#still spherical structure
#random allocation rather than initial lattice structure 
def two_particle_system_2D(L, N_total, particle_1_fraction, area_fraction, radius_scale_factor):
    n1 = int(N_total * particle_1_fraction)
    n2 = N_total - n1
    #calculate radius of each particle type 
    area_1 = area_fraction * (L**2) / (n1 + (radius_scale_factor**2) * n2)
    radius_1 = (area_1 / np.pi) ** 0.5
    radius_2 = radius_scale_factor * radius_1
    #generate random positions 
    #first list in positions is particle type 1, second list is particle type 2
    positions = [[], []]  # lists of [x, y]

    # ---- Insert type 1 particles ----
    while len(positions[0]) < n1:
        pos = np.array([random.uniform(0, L), random.uniform(0, L)])

        if len(positions[0]) > 0:
            diffs = np.array(positions[0]) - pos
            diffs -= L * np.round(diffs / L)
            dists = np.linalg.norm(diffs, axis=1)
            if not np.all(dists >= 2 * radius_1):
                continue

        positions[0].append(pos)

    # ---- Insert type 2 particles ----
    while len(positions[1]) < n2:
        pos = np.array([random.uniform(0, L), random.uniform(0, L)])

        d1 = np.array(positions[0]) - pos
        d1 -= L * np.round(d1 / L)
        if not np.all(np.linalg.norm(d1, axis=1) >= radius_1 + radius_2):
            continue

        if len(positions[1]) > 0:
           d2 = np.array(positions[1]) - pos
           d2 -= L * np.round(d2 / L)
           if not np.all(np.linalg.norm(d2, axis=1) >= 2 * radius_2):
               continue

        positions[1].append(pos)

    return np.array(positions[0]), np.array(positions[1]), radius_1, radius_2




In [19]:
#create plot to visualise two particle system
L = 1
N_total = 100
particle_1_fraction = 0.7
area_fraction = 0.4
radius_scale_factor = 0.7
positions = [[],[]] # output positions into single lits containing both particle types 
positions[0], positions[1], r1, r2 = two_particle_system_2D(L, N_total, particle_1_fraction, area_fraction, radius_scale_factor)


x1, y1 = zip(*positions[0])
x2, y2 = zip(*positions[1]) 
plt.figure()
plt.scatter(x1, y1, label = 'Particle Type 1', alpha = 0.5, s= 120)
plt.scatter(x2, y2, label = 'Particle Type 2', alpha = 0.5, s =  120*(0.7**2))
plt.xlim(0,L)
plt.ylim(0,L)
plt.savefig('two_particle_system_2D.png')

In [20]:
#alter the original monte carlo method to accept two different sized particles
def monte_carlo_two_particle_2D(pos1, pos2, d_max, L, r1, r2):
    N1 = len(pos1)
    N2 = len(pos2)
    N = N1 + N2
    pos1 = np.asarray(pos1)
    pos2 = np.asarray(pos2)
    positions = np.vstack((pos1, pos2))
    new_positions = positions.copy()
    # pick particle
    particle = random.randrange(N)
    #n will reduce the displacement randomly
    n = random.random()
    # trial displacement
    theta = random.uniform(0, 2*np.pi)
    x_displacement = d_max * np.cos(theta)*n
    y_displacement = d_max * np.sin(theta)*n
    
    # --------------------
    # Move particle of type 1
    # --------------------

    x_old, y_old = positions[particle]
    new_positions[particle,0] += x_displacement
    new_positions[particle,1] += y_displacement
    new_positions[particle] %=L
        # type-1 vs type-1
    diffs = new_positions - new_positions[particle]
    diffs = diffs - L * np.round(diffs / L)
    distances = np.linalg.norm(diffs,axis=1)
    distances[particle] = np.inf  # ignore self-distance

    if particle < N1:
        if np.any(distances[0:particle] < 2*r1) or np.any(distances[particle:N1] < 2*r1) or np.any(distances[N1:] < r1+r2):
            return pos1.copy(), pos2.copy() #reject move:
        else:
            return new_positions[0:N1], new_positions[N1:] #accept move
    else:
        if np.any(distances[0:N1]<r1+r2) or np.any(distances[N1:particle]<2*r2) or np.any(distances[particle:] <2*r2):
            return pos1.copy(), pos2.copy() #reject move
        else:      
            return new_positions[0:N1], new_positions[N1:] #accept move


In [21]:
#define two particle initial system in 3D 
def two_particle_system_3D(L, N_total, particle_1_fraction, vol_fraction, radius_scale_factor):
    n1 = int(N_total * particle_1_fraction)
    n2 = N_total - n1
    #calculate radius of each particle type
    vol_1 = vol_fraction * (L**3) / (n1 + (radius_scale_factor**3) * n2)
    r_1 = ( (3/4) * (vol_1 / np.pi) ) ** (1/3)
    r_2 = radius_scale_factor * r_1
    #generate random positions
    positions = [[],[]] # empty initial positions lists for x,y,z coordinates
    while len(positions[0]) < n1:
        pos = np.array([random.uniform(0, L), random.uniform(0, L), random.uniform(0,L)])

        if len(positions[0]) > 0:
            diffs = np.array(positions[0]) - pos
            dists = np.linalg.norm(diffs, axis=1)
            if not np.all(dists >= 2 * r_1):
                continue
 
        positions[0].append(pos)
    while len(positions[1]) < n2:
        pos = np.array([random.uniform(0, L), random.uniform(0, L), random.uniform(0,L)])

        if len(positions[0]) > 0:
            d1 = np.linalg.norm(np.array(positions[0]) - pos, axis=1)
            if not np.all(d1 >= r_1 + r_2):
                continue

        if len(positions[1]) > 0:
            d2 = np.linalg.norm(np.array(positions[1]) - pos, axis=1)
            if not np.all(d2 >= 2 * r_2):
                continue

        positions[1].append(pos)

    return np.array(positions[0]), np.array(positions[1]), r_1, r_2



In [22]:
#try monte carlo run for 2 particle 2D system - think the original pair correlation function should work
#use the results from previous code block for initial positions
L=1 
N_total = 100
particle_1_fraction = 0.7
area_fraction = 0.55
radius_scale_factor = 0.7
positions = [[],[]]
positions[0], positions[1], r1, r2 = two_particle_system_2D(L, N_total, particle_1_fraction, area_fraction, radius_scale_factor)

#find monte carlo step size for reasonable acceptance rate 
acceptance_count = 0
for i in range(0,1000):
    pos1_new, pos2_new = monte_carlo_two_particle_2D(positions[0], positions[1], 0.02, L, r1, r2)
    if np.array_equal(pos1_new, positions[0]) and np.array_equal(pos2_new, positions[1]):
        pass #move rejected
    else:
        acceptance_count += 1 #move accepted
        positions[0] = pos1_new
        positions[1] = pos2_new
print(acceptance_count)


617


In [23]:
#after acceptance count is checked, run monte carlo to equillibrate 
positions = [[],[]]
positions[0], positions[1], r1, r2 = two_particle_system_2D(L, N_total, particle_1_fraction, area_fraction, radius_scale_factor)
#equillibration run
for i in range(0, 8*10 **5 + 1):
    positions[0], positions[1] = monte_carlo_two_particle_2D(np.array(positions[0]),np.array(positions[1]), 0.02, L, r1, r2)

In [24]:
#monte carlo equillibrated so start with pair correlation function calculation for two particle system
#total pair correlation function and also pair correlation for each particle type separately
positions_2D = [positions[0].copy(), positions[1].copy()]

d1 = 2 * r1
d2 = 2 * r2
n1 = int(N_total * particle_1_fraction)
n2 = N_total - n1
average_diameter = (n1*d1 + n2*d2) / N_total

g_r_total_av = []
g_r_total_1 = []
g_r_total_2 = []

for step in range(5*10**4 + 1):

    positions_2D[0], positions_2D[1] = monte_carlo_two_particle_2D(np.array(positions_2D[0]), np.array(positions_2D[1]), 0.02, L, r1, r2)

    if step % 1000 == 0:
        # TOTAL g(r)
        positions_total = np.vstack((positions_2D[0], positions_2D[1]))
        r_total, g_r_total = pair_correlation_2D(positions_total, average_diameter, L, bins=95)
        g_r_total_av.append(g_r_total)

        # TYPE 1
        r_1, g_r_1 = pair_correlation_2D(positions_2D[0], d1, L, bins=95)
        g_r_total_1.append(g_r_1)

        # TYPE 2
        r_2, g_r_2 = pair_correlation_2D(positions_2D[1], d2, L, bins=95)
        g_r_total_2.append(g_r_2)



In [25]:
#plot graph of the pair correlation functions
x_1_2D = r_1
x_2_2D = r_2
x_total_2D = r_total
y_1_2D = np.mean(g_r_total_1, axis=0)
y_2_2D = np.mean(g_r_total_2, axis=0)
y_total_2D = np.mean(g_r_total_av, axis=0)

plt.figure()
plt.plot(x_total_2D, y_total_2D, color = 'black', linewidth = 0.6, label = 'Total g(r)')
plt.plot(x_1_2D, y_1_2D, color = 'blue', linewidth = 0.6, label = 'Particle Type 1 g(r)')
plt.plot(x_2_2D, y_2_2D, color = 'red', linewidth = 0.6, label = 'Particle Type 2 g(r)')
plt.legend()
plt.xlabel('r / $\sigma$')
plt.ylabel('g(r)')
plt.savefig('two_particle_g(r)_2D.png')


<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_28206/254346491.py:14: SyntaxWarning: invalid escape sequence '\s'
  plt.xlabel('r / $\sigma$')


In [26]:
x1, y1 = zip(*positions_2D[0])
x2, y2 = zip(*positions_2D[1])
plt.figure()
plt.scatter(x1, y1, label = 'Particle Type 1', alpha = 0.5, s = 550)
plt.scatter(x2, y2, label = 'Particle Type 2', alpha = 0.5, s =  550*(radius_scale_factor**2))
plt.xlim(0,L)
plt.ylim(0,L)
plt.title('Equillibrated Two Particle System (Area Fraction = 0.55)')
plt.savefig('two_particle_system_2D_equillibrated.png')
plt.legend()


In [27]:
#change shape of particle for phospholipid element
def phospholipid_water_2D(L, N_total,water_fraction, head_radius_ratio,c_ratio, n_c_per_chain, area_fraction):
    #water_fraction is fraction of total particles that are water
    #head_radius_ratio is ratio of water particle radius to phospholipid head radius
    #c_ratio is ratio of water particle radius to carbon atom diameter in tail
    #area_fraction is total area fraction of all particles in system
    n_water = int(N_total * water_fraction)
    n_phospholipid = N_total - n_water
    n_head = n_phospholipid
    n_tail = n_phospholipid
    area_water = area_fraction * (L**2) / (n_water + head_radius_ratio**2 * n_head + (c_ratio**2 * n_c_per_chain * n_tail))
    r_water = (area_water / np.pi) ** 0.5
    r_head = head_radius_ratio * r_water
    r_carbon = c_ratio * r_water
    positions = [[],[],[]] 
    max_tries = 100000
    tries = 0
    # lists for heads, tails, water [list for tails will be a list with position of each carbon atom in tail - 2 lists per phosphate head]
    # ---- Insert phospholipids first ----
    # Insert heads at same time as tail
    while len(positions[0]) < n_head and tries < max_tries:
        tries += 1
        if tries % 1000 ==0:
            print(f'Tries = {tries}, Phospholipids placed = {len(positions[0])}/{n_head}')
        tail_positions = [] # xcoords and ycoords list for carbon atoms in tails
        head_pos = np.array([random.uniform(0, L), random.uniform(0, L)])
        angle = random.uniform(0, 2*np.pi)
        tail_x = (head_pos[0] + np.cos(angle)*(r_head + r_carbon))%L
        tail_y = (head_pos[1] + np.sin(angle)*(r_head + r_carbon))%L

        tail_positions.append(np.array([tail_x, tail_y]))
        while len(tail_positions) < n_c_per_chain:
            angle += np.random.normal(0, np.pi/16) # add some flexibility to tail correlates to real flexibility of phospholipid tails
            tail_x = (tail_positions[-1][0] + 2 * np.cos(angle)*r_carbon)%L
            tail_y = (tail_positions[-1][1] + 2* np.sin(angle)*r_carbon)%L
            tail_positions.append(np.array([tail_x, tail_y])) # tail positions is list of n carbon atoms lists with x and y coords
             #periodic boundary conditions
        #check for overlaps with existing particles
        if len(positions[0]) > 0:
            diffs_heads = np.array(positions[0]) - head_pos # check head - head overlap
            diffs_heads -= L * np.round(diffs_heads / L)
            dists_heads = np.linalg.norm(diffs_heads, axis=1)
            if not np.all(dists_heads >= 2 * r_head):
                continue
            #check head-tail overlaps
            overlap = False
            for existing_tail in positions[1]:
                diffs = np.array(existing_tail) - head_pos
                diffs -= L * np.round(diffs / L)
                if np.any(np.linalg.norm(diffs, axis=1) < (r_head + r_carbon)):
                    overlap = True
                    break
            if overlap:
                continue
            #check tail - head overlaps
            diffs_tail_head_total = []
            for tail_pos in tail_positions:
                diffs_tail_head = np.array(positions[0]) - np.array(tail_pos)
                diffs_tail_head -=L * np.round(diffs_tail_head / L)
                dists_tail_head = list(np.linalg.norm(diffs_tail_head, axis=1))
                diffs_tail_head_total.append(dists_tail_head)
            if not np.all(np.array(diffs_tail_head_total) >= r_head + r_carbon):
                continue
            #check tail - tail overlaps
            
            overlap = False
            for tail_pos in tail_positions:
                for existing_tail in positions[1]:
                    diffs = np.array(existing_tail) - tail_pos
                    diffs -= L * np.round(diffs / L)
                    if np.any(np.linalg.norm(diffs, axis=1) < 2 * r_carbon):
                        overlap = True
                        break
                if overlap:
                    break

            if overlap:
                continue
        positions[0].append(head_pos)
        positions[1].append(tail_positions)
    tries = 0
    #Now time to insert the water molecules
    while len(positions[2]) < n_water and tries < max_tries:
        water_pos = np.array([random.uniform(0, L), random.uniform(0, L)])
        tries += 1
        if tries % 1000 == 0:
            print(f'Tries for water = {tries}, Water placed = {len(positions[2])}/{n_water}')
        
        overlap = False
        #check for overlaps with all phospholipids
        diffs_water_head = np.array(positions[0]) - water_pos
        diffs_water_head -= L * np.round(diffs_water_head / L)
        dists_water_head = np.linalg.norm(diffs_water_head, axis=1)
        if not np.all(dists_water_head >= r_head + r_water):
            continue
        diffs_water_tail_total = []
        for existing_tail in positions[1]:
            diffs = np.array(existing_tail) - water_pos
            diffs -= L * np.round(diffs / L)
            if np.any(np.linalg.norm(diffs, axis=1) < (r_water + r_carbon)):
                overlap = True
                break
        if overlap:
            continue
        #check water - water overlaps
        if len(positions[2]) > 0:
            diffs_water_water = np.array(positions[2]) - water_pos
            diffs_water_water -= L * np.round(diffs_water_water / L)
            dists_water_water = np.linalg.norm(diffs_water_water, axis=1)
            if not np.all(dists_water_water >= 2 * r_water):
                continue
        positions[2].append(water_pos)
    
    
    return np.array(positions[0]), np.array(positions[1]), np.array(positions[2]), r_water, r_head, r_carbon

In [28]:
#trial placing a phospholipid system
L = 1
N_total = 1000
water_fraction = 0.95
head_radius_ratio = 1.5
c_ratio = 0.7
n_c_per_chain = 4
area_fraction = 0.4
positions_heads, positions_tails, positions_water, r_water, r_head, r_carbon = phospholipid_water_2D(L, N_total, water_fraction, head_radius_ratio, c_ratio, n_c_per_chain, area_fraction)
#plot the phospholipid water system

x_heads, y_heads = zip(*positions_heads)
x_tails = []
y_tails = []
for tail in positions_tails:
    x_tail, y_tail = zip(*tail)
    x_tails.extend(x_tail)
    y_tails.extend(y_tail)
    
plt.figure()
from matplotlib.patches import Circle

fig, ax = plt.subplots()
ax.set_aspect('equal')

for h in positions_heads:
    ax.add_patch(Circle(h, r_head, fill=True, fc = 'red', alpha =1))

for tail in positions_tails:
    for t in tail:
        ax.add_patch(Circle(t, r_carbon, fill=True, fc = 'orange'))
for w in positions_water:
    ax.add_patch(Circle(w, r_water, fill=True, fc = 'blue'))

ax.set_xlim(0, L)
ax.set_ylim(0, L) #this does work for no overlaps on graph
plt.savefig('phospholipid_trial_2D') 
#should have working model for phospholipid and water placing in the original structure
# flatten all particles

Tries for water = 1000, Water placed = 496/950
Tries for water = 2000, Water placed = 687/950
Tries for water = 3000, Water placed = 799/950
Tries for water = 4000, Water placed = 890/950


In [29]:
#time to define the metropolis algorithm then the monte carlo algorithm for the phospholipid system 
#check for overlaps in monte carlo - metropolis algorithm will be used to accept or reject monte carlo steps based on change to energy as hard
#sphere model will always reject with any overlaps
#need to define n_c_per_chain before running this code

import math
from numba import njit

@njit
def monte_carlo_wp_2D(positions_heads, positions_tails, positions_water,
                      d_max, rotation_max, L, r_water, r_head, r_carbon,
                      n_c_per_chain):

    N_heads = positions_heads.shape[0]
    N_water = positions_water.shape[0]
    
    particle = int(np.random.rand() * (N_heads + N_water))

    displacement = d_max * np.random.normal()
    angle = 2.0 * math.pi * np.random.rand()
    x_displacement = displacement * np.cos(angle)
    y_displacement = displacement * np.sin(angle)

    rotation_angle = 2.0 * math.pi * np.random.rand()
    vec = positions_tails[particle, 0] - positions_heads[particle]
    initial_angle = np.arctan2(vec[1], vec[0])
    # ==========================================================
    # PHOSPHOLIPID MOVE
    # ==========================================================
    if particle < N_heads:

        moved_index = particle
        moved_type = 0

        pos_head = positions_heads[particle]

        # ---------- CHANGED: use NumPy array ----------
        new_pos_head = np.array([
            (pos_head[0] + x_displacement) % L,
            (pos_head[1] + y_displacement) % L
        ])

        # ---------- CHANGED: preallocate tail ----------
        new_tail_positions = np.empty((n_c_per_chain, 2))
        #need to find an initial angle so no preferential direction 

        new_tail_positions[0, 0] = (new_pos_head[0] + np.cos(rotation_angle) * (r_head + r_carbon)) % L
        new_tail_positions[0, 1] = (new_pos_head[1] + np.sin(rotation_angle) * (r_head + r_carbon)) % L

        for i in range(1, n_c_per_chain):

            prev_x = new_tail_positions[i-1, 0]
            prev_y = new_tail_positions[i-1, 1]

            if i == 1:
                ref_x, ref_y = new_pos_head
            else:
                ref_x = new_tail_positions[i-2, 0]
                ref_y = new_tail_positions[i-2, 1]

            initial_angle = math.atan2(prev_y - ref_y, prev_x - ref_x)
            new_angle = initial_angle + (rotation_max / 4.0) * np.random.normal()

            new_tail_positions[i, 0] = (prev_x + 2 * r_carbon * math.cos(new_angle)) % L
            new_tail_positions[i, 1] = (prev_y + 2 * r_carbon * math.sin(new_angle)) % L

        # ---------- overlap checks ----------

        # head–water
        for i in range(N_water):
            dx = positions_water[i, 0] - new_pos_head[0]
            dy = positions_water[i, 1] - new_pos_head[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (r_head + r_water)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # head–head
        for j in range(N_heads):
            if j == particle:
                continue
            dx = positions_heads[j, 0] - new_pos_head[0]
            dy = positions_heads[j, 1] - new_pos_head[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (2 * r_head)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # head–tail
        for j in range(N_heads):
            if j == particle:
                continue
            for k in range(n_c_per_chain):
                dx = positions_tails[j, k, 0] - new_pos_head[0]
                dy = positions_tails[j, k, 1] - new_pos_head[1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_head + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # tail–water
        for k in range(n_c_per_chain):
            for i in range(N_water):
                dx = positions_water[i, 0] - new_tail_positions[k, 0]
                dy = positions_water[i, 1] - new_tail_positions[k, 1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_water + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        # tail–tail
        for j in range(N_heads):
            if j == particle:
                continue
            for k in range(n_c_per_chain):
                for m in range(n_c_per_chain):
                    dx = positions_tails[j, m, 0] - new_tail_positions[k, 0]
                    dy = positions_tails[j, m, 1] - new_tail_positions[k, 1]
                    dx -= L * round(dx / L)
                    dy -= L * round(dy / L)
                    if dx*dx + dy*dy < (2 * r_carbon)**2:
                        return positions_heads, positions_tails, positions_water, False, moved_index, moved_type
        #tail - heads
        for j in range(N_heads):
            for k in range(n_c_per_chain):
                dx = positions_heads[j, 0] - new_tail_positions[k, 0]
                dy = positions_heads[j, 1] - new_tail_positions[k, 1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_head + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type
        # ---------- accept ----------
        positions_heads[particle] = new_pos_head
        positions_tails[particle] = new_tail_positions

        return positions_heads, positions_tails, positions_water, True, moved_index, moved_type

    # ==========================================================
    # WATER MOVE (unchanged logic, array-safe)
    # ==========================================================
    else:
        particle -= N_heads
        moved_index = particle
        moved_type = 1

        pos_water = positions_water[particle]

        new_pos_water = np.array([
            (pos_water[0] + x_displacement) % L,
            (pos_water[1] + y_displacement) % L
        ])

        for i in range(N_water):
            if i == particle:
                continue
            dx = positions_water[i, 0] - new_pos_water[0]
            dy = positions_water[i, 1] - new_pos_water[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (2 * r_water)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        for j in range(N_heads):
            dx = positions_heads[j, 0] - new_pos_water[0]
            dy = positions_heads[j, 1] - new_pos_water[1]
            dx -= L * round(dx / L)
            dy -= L * round(dy / L)
            if dx*dx + dy*dy < (r_water + r_head)**2:
                return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

            for k in range(n_c_per_chain):
                dx = positions_tails[j, k, 0] - new_pos_water[0]
                dy = positions_tails[j, k, 1] - new_pos_water[1]
                dx -= L * round(dx / L)
                dy -= L * round(dy / L)
                if dx*dx + dy*dy < (r_water + r_carbon)**2:
                    return positions_heads, positions_tails, positions_water, False, moved_index, moved_type

        positions_water[particle] = new_pos_water
        return positions_heads, positions_tails, positions_water, True, moved_index, moved_type
count = 0
for i in range(1000):
    positions_heads, positions_tails, positions_water, accepted, moved_index, moved_type = monte_carlo_wp_2D(
        positions_heads, positions_tails, positions_water,
        d_max=0.02, rotation_max=np.pi/8, L=L,
        r_water=r_water, r_head=r_head, r_carbon=r_carbon,
        n_c_per_chain=n_c_per_chain
    )
    if accepted:
        count += 1
print(f"Total accepted moves: {count}")

Total accepted moves: 351


In [30]:
#time to define the metropolis algorithm for the phospholipid system 
#only want to consider nearest neighbour interactions for energy calculations
#only need to consider the energy change ffor the metropolis algorithm 

@njit
def metropolis_pw_2D(positions_heads, positions_tails, positions_water, L, r_water, r_head, r_carbon, epsilon, Temperature, d_max, n_c_per_chain):
    """
    Metropolis algorithm for phospholipid system in 2D.
    Only nearest-neighbour interactions are considered.
    Assumes positions are numpy arrays:
    positions_heads: (N_heads, 2)
    positions_tails: (N_heads, n_c_per_chain, 2)
    positions_water: (N_water, 2)
    """
    N_heads = positions_heads.shape[0]
    N_water = positions_water.shape[0]
    
    # Make copies for trial moves
    pos_heads = np.copy(positions_heads)
    pos_tails = np.copy(positions_tails)
    pos_water = np.copy(positions_water)

    # Perform MC move (already njit)
    new_heads, new_tails, new_water, accepted, moved_index, moved_type = monte_carlo_wp_2D(
        positions_heads, positions_tails, positions_water, d_max, np.pi/32, 
        L, r_water, r_head, r_carbon, n_c_per_chain
    )

    if not accepted:
        return positions_heads, positions_tails, positions_water, False, moved_type

    # Initialize energies
    initial_energy = 0.0
    final_energy = 0.0

    # --- Head particle moved ---
    if moved_type == 0:
        # Tail-water interactions
        for c in range(n_c_per_chain):
            tail_old = pos_tails[moved_index, c]
            tail_new = new_tails[moved_index, c]

            # New
            diffs_new = new_water - tail_new
            diffs_new -= L * np.round(diffs_new / L)
            dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
            for i in range(dists_new.shape[0]):
                if dists_new[i] < 1.5 * (r_carbon + r_water):
                    final_energy += epsilon

            # Old
            diffs_old = pos_water - tail_old
            diffs_old -= L * np.round(diffs_old / L)
            dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
            for i in range(dists_old.shape[0]):
                if dists_old[i] < 1.5 * (r_carbon + r_water):
                    initial_energy += epsilon

        # Tail-tail interactions
        for c in range(n_c_per_chain):
            tail_new = new_tails[moved_index, c]
            tail_old = pos_tails[moved_index, c]

            for j in range(N_heads):
                if j == moved_index:
                    continue
                for cc in range(n_c_per_chain):
                    # New
                    diff = new_tails[j, cc] - tail_new
                    diff -= L * np.round(diff / L)
                    dist = np.sqrt(diff[0]**2 + diff[1]**2)
                    if dist < 1.5 * (r_carbon + r_carbon):
                        final_energy -= epsilon
                    # Old
                    diff_old = pos_tails[j, cc] - tail_old
                    diff_old -= L * np.round(diff_old / L)
                    dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
                    if dist_old < 1.5 * (r_carbon + r_carbon):
                        initial_energy -= epsilon
        #head - water interactions
        diffs_new = new_water - new_heads[moved_index]
        diffs_new -= L * np.round(diffs_new / L)
        dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
        for i in range(dists_new.shape[0]):
            if dists_new[i] < 1.5 * (r_head + r_water):
                final_energy += epsilon
        diffs_old = pos_water - pos_heads[moved_index]
        diffs_old -= L * np.round(diffs_old / L)
        dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
        for i in range(dists_old.shape[0]):
            if dists_old[i] < 1.5 * (r_head + r_water):
                initial_energy += epsilon
    # --- Water particle moved ---
    else:
        water_old = pos_water[moved_index]
        water_new = new_water[moved_index]

        # Water-water interactions
        for i in range(N_water):
            if i == moved_index:
                continue
            # New
            diff = new_water[i] - water_new
            diff -= L * np.round(diff / L)
            dist = np.sqrt(diff[0]**2 + diff[1]**2)
            if dist < 1.5 * (r_water + r_water):
                final_energy -= epsilon
            # Old
            diff_old = pos_water[i] - water_old
            diff_old -= L * np.round(diff_old / L)
            dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
            if dist_old < 1.5 * (r_water + r_water):
                initial_energy -= epsilon

        # Water-tail interactions
        for i in range(N_heads):
            for c in range(n_c_per_chain):
                # New
                diff = new_tails[i, c] - water_new
                diff -= L * np.round(diff / L)
                dist = np.sqrt(diff[0]**2 + diff[1]**2)
                if dist < 1.5 * (r_carbon + r_water):
                    final_energy += epsilon
                # Old
                diff_old = pos_tails[i, c] - water_old
                diff_old -= L * np.round(diff_old / L)
                dist_old = np.sqrt(diff_old[0]**2 + diff_old[1]**2)
                if dist_old < 1.5 * (r_carbon + r_water):
                    initial_energy += epsilon

        #head - water interactions
        diffs_new = new_heads - water_new
        diffs_new -= L * np.round(diffs_new / L)
        dists_new = np.sqrt(diffs_new[:,0]**2 + diffs_new[:,1]**2)
        for i in range(dists_new.shape[0]):
            if dists_new[i] < 1.5 * (r_head + r_water):
                final_energy += epsilon
        diffs_old = pos_heads - water_old
        diffs_old -= L * np.round(diffs_old / L)
        dists_old = np.sqrt(diffs_old[:,0]**2 + diffs_old[:,1]**2)
        for i in range(dists_old.shape[0]):
            if dists_old[i] < 1.5 * (r_head + r_water):
                initial_energy += epsilon

    # Metropolis criterion
    delta_E = final_energy - initial_energy
    if delta_E < 0.0 or np.exp(-delta_E / (k_b * Temperature)) >= np.random.rand():
        return new_heads, new_tails, new_water, True, moved_type
    else:
        return pos_heads, pos_tails, pos_water, False, moved_type
    

In [31]:
#bug testing the metropolis algorithm for phospholipid system 
L = 1
N_total = 500
water_fraction = 0.8
head_radius_ratio = 1.1
c_ratio = 0.4
n_c_per_chain = 6 
area_fraction = 0.35
positions_heads, positions_tails, positions_water, r_water, r_head, r_carbon = phospholipid_water_2D(L, N_total, water_fraction, head_radius_ratio, c_ratio, n_c_per_chain, area_fraction)

t = 2 # ratio from the paper found for energy and temperature 
#t = k_b * T / epsilon
epsilon = 1
k_b = 1.4 * 10 ** -23
Temperature = t * epsilon / k_b
d_max = r_water / 10
count = 0
water_count = 0
head_count = 0
for i in range(1000):
    positions_heads, positions_tails, positions_water, accepted, moved_type = metropolis_pw_2D(positions_heads, positions_tails, positions_water, L, r_water, r_head, r_carbon, epsilon, Temperature, d_max, n_c_per_chain)
    if accepted:
        count += 1
        if moved_type ==1:
            water_count += 1
        else:
            head_count += 1
             
print(f'Acceptance count for metropolis algorithm = {count} out of 1000 moves')
print(f'Water count = {water_count}')
print(f'Head count = {head_count}')

plt.figure()
from matplotlib.patches import Circle

fig, ax = plt.subplots()
ax.set_aspect('equal')

for h in positions_heads:
    ax.add_patch(Circle(h, r_head, fill=True, fc = 'red', alpha =1))

for tail in positions_tails:
    for t in tail:
        ax.add_patch(Circle(t, r_carbon, fill=True, fc = 'orange'))
for w in positions_water:
    ax.add_patch(Circle(w, r_water, fill=True, fc = 'blue'))

ax.set_xlim(0, L)
ax.set_ylim(0, L) #this does work for no overlaps on graph
plt.savefig('trial_1.png')
plt.figure()
from matplotlib.patches import Circle
#seems to be working for the system

Tries for water = 1000, Water placed = 281/400
Tries for water = 2000, Water placed = 372/400
Acceptance count for metropolis algorithm = 719 out of 1000 moves
Water count = 702
Head count = 17


In [33]:

N_total = 1000
water_fraction = 0.8
area_fraction = 0.4
head_radius_ratio = 1.1
c_ratio = 0.4
n_c_per_chain = 6
d_max = r_water/3
positions_heads, positions_tails, positions_water, r_water, r_head, r_carbon = phospholipid_water_2D(L, N_total, water_fraction, head_radius_ratio, c_ratio, n_c_per_chain, area_fraction)
#trial run at higher number of moves
count = 0
for i in range(10000000):
    positions_heads, positions_tails, positions_water, accepted, moved_type = metropolis_pw_2D(positions_heads, positions_tails, positions_water, L, r_water, r_head, r_carbon, epsilon, Temperature, d_max, n_c_per_chain)
    if accepted:
        count += 1
    if i%1000 == 0:
        print(f'Move {i}, Accepted moves so far: {count}')

Tries for water = 1000, Water placed = 318/800
Tries for water = 2000, Water placed = 478/800
Tries for water = 3000, Water placed = 590/800
Tries for water = 4000, Water placed = 661/800
Tries for water = 5000, Water placed = 722/800
Tries for water = 6000, Water placed = 768/800
Move 0, Accepted moves so far: 1
Move 1000, Accepted moves so far: 583
Move 2000, Accepted moves so far: 1157
Move 3000, Accepted moves so far: 1727
Move 4000, Accepted moves so far: 2284
Move 5000, Accepted moves so far: 2847
Move 6000, Accepted moves so far: 3396
Move 7000, Accepted moves so far: 3927
Move 8000, Accepted moves so far: 4475
Move 9000, Accepted moves so far: 5037
Move 10000, Accepted moves so far: 5590
Move 11000, Accepted moves so far: 6156
Move 12000, Accepted moves so far: 6708
Move 13000, Accepted moves so far: 7238
Move 14000, Accepted moves so far: 7779
Move 15000, Accepted moves so far: 8329
Move 16000, Accepted moves so far: 8899
Move 17000, Accepted moves so far: 9422
Move 18000, Acc

In [ ]:
plt.figure()
from matplotlib.patches import Circle

fig, ax = plt.subplots()
ax.set_aspect('equal')

for h in positions_heads:
    ax.add_patch(Circle(h, r_head, fill=True, fc = 'red', alpha =1))

for tail in positions_tails:
    for t in tail:
        ax.add_patch(Circle(t, r_carbon, fill=True, fc = 'orange'))
for w in positions_water:
    ax.add_patch(Circle(w, r_water, fill=True, fc = 'blue'))

ax.set_xlim(0, L)
ax.set_ylim(0, L) #this does work for no overlaps on graph
plt.savefig('trial_1.png')
#trial 1 is 10,000,000 moves with 1000 particles 


In [36]:
#if metropolis algorithm is working - need to figure out what to calculate to show micelle formation 
#calculate pair correlation function for phospholipids should see peaks where there are clusters of micelles
def phospholipid_pair_correlation_2D(positions, r_water, r_head, r_carbon, L, bins):
    #positions is list positions = [heads, tails, water]
    #calculate pair correlation function for phospholipid heads and tails separately
    #only consider head - head interactions and tail - tail interactions for pair correlation function calculations
    #bins is number of bins for pair correlation function calculation
    #calculate g(r) for heads
    head_positions = np.array(positions[0])
    tail_positions = np.array(positions[1]).reshape(-1, 2) # reshape tail positions to be a list of all tail particle positions
    r_head_head, g_r_head_head = pair_correlation_2D(head_positions, r_head*2, L, bins)
    r_tail_tail, g_r_tail_tail = pair_correlation_2D(tail_positions, r_carbon*2, L, bins)
    return r_head_head, g_r_head_head, r_tail_tail, g_r_tail_tail

#plot the positions found above 
positions = [positions_heads, positions_tails, positions_water]
r_head_head, g_r_head_head, r_tail_tail, g_r_tail_tail = phospholipid_pair_correlation_2D(positions, r_water, r_head, r_carbon, L, bins=95)
plt.figure()
plt.plot(r_head_head, g_r_head_head, color = 'red', linewidth = 0.6, label = 'Head-Head g(r)')
plt.plot(r_tail_tail, g_r_tail_tail, color = 'orange', linewidth = 0.6, label = 'Tail-Tail g(r)')
plt.xlabel('Distance r')
plt.ylabel('g(r)')
plt.legend()
plt.savefig('phospholipid_g(r)_10000_moves.png')



/tmp/ipykernel_28206/2515550907.py:18: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  plt.figure()
